# Read mapping and depth (density) calculation

for RNA-Seq/Ribo-Seq data of Davis, Gohara & Yap (PNAS, 2014)
- See [this notebook](../00-prep-pubdata/210615_expression_DGY14PNAS.ipynb) for procedures to download the data.

- For both RNA- and Ribo-Seq, adapter sequences were automatically detected and trimmed from raw fastq reads by fastp (v0.20.1). Then bowtie2 (v2.4.4) was used to map the trimmed reads to the reference genomes (see [metadata](../../metadata/expression/DGY14PNAS_refs.tsv)). Both tools was executed under the default parameters.


- For RNA-Seq, position-wise read depths were counted, ignoring multi-mapped reads and reads that mapped to tRNA/rRNA on the same strand.
- For Ribo-Seq, position-wise ribosome read densities were calculated by assigning reads to the 3' end positions. Multi-mapped reads and reads that mapped to tRNA/rRNA on same strand were also removed.

## Docker condifuration

docker-compose.yml
```
version: '3'
services:
  (...)
  mapping:
    build: ./docker/mapping
    volumes:
      - ./data:/data
      - ./metadata:/metadata
      - ./scripts:/scripts
  (...)
```

docker/mapping/Dockerfile
```
FROM continuumio/miniconda3:4.9.2
RUN conda install -c conda-forge -y mamba==0.8.2
RUN mamba install -c conda-forge -c bioconda -y \
        sra-tools==2.11.0 fastp==0.20.1 bowtie2==2.4.4 samtools==1.12  
```

## Commands and scripts to reproduce reads mapping

```
docker-compose run --rm mapping /bin/bash scripts/210615_2_DGY14PNAS.sh
```

scripts/210615_2_DGY14PNAS.sh:

```{bash}
#!/bin/bash -eu

# reference genome indexing
run_bowtie2build () {
    local infile=$1
    local outfile=${infile/reference/bowtie2_idx}
    local logfile=${outfile/.fna/.log}
    outfile=${outfile/.fna/}
    
    bowtie2-build $infile $outfile &> $logfile
}

export -f run_bowtie2build

ls /data/expression/DGY14PNAS/reference/*.fna | xargs -t -P5 -L1 -I{} /bin/bash -c 'run_bowtie2build {}'

# adapter sequence trimming
run_fastp () {
    local infile=$1
    local outfile=${infile/pubdata\//}
    outfile=${outfile/fastq/trimmed_fastq}
    local htmlfile=${outfile/.fastq.gz/.html}
    local jsonfile=${outfile/.fastq.gz/.json}
    local logfile=${outfile/.fastq.gz/.log}
    outfile=${outfile/.fastq.gz/.trimmed.fastq.gz}
    
    fastp -i $infile -a CTGTAGGCACCATCAAT -o $outfile -h $htmlfile -j $jsonfile 2> $logfile
}

export -f run_fastp

ls /data/pubdata/expression/DGY14PNAS/fastq/*.fastq.gz | xargs -t -P40 -L1 -I{} /bin/bash -c 'run_fastp {}'

# read mapping
run_bowtie2align () {
    local basename=/data/expression/DGY14PNAS
    local readsfile=$basename/trimmed_fastq/$1.trimmed.fastq.gz
    local reffile=$basename/bowtie2_idx/$2
    local outfile=$basename/bam/$1.bam
    local logfile=$basename/bam/$1.log
    bowtie2 --very-sensitive --no-unal -x $reffile -U $readsfile -p 8 2> $logfile | samtools view -b | samtools sort -o $outfile -@ 8 
}

export -f run_bowtie2align

tail -n +2 /metadata/expression/DGY14PNAS_runs.tsv | awk '{print $1,$12}' | xargs -t -P12 -L2 -I{} /bin/bash -c 'run_bowtie2align {}'


```

### Depth calculation

In [1]:
import os
import pandas as pd
import pysam
import tempfile
from multiprocessing import Pool
from tqdm.notebook import tqdm
from pathlib import Path
from pyscripts.config import path2
from pyscripts.datasets import DatasetLoader
from pyscripts.genomeutil import get_intervaltree
dloader = DatasetLoader()

In [2]:
refs = pd.read_csv(path2.metadata/'expression'/'DGY14PNAS_refs.tsv', sep='\t')
runs = pd.read_csv(path2.metadata/'expression'/'DGY14PNAS_runs.tsv', sep='\t')

In [3]:
ignore_intervals = {
    ref: {
        rec.id:
        get_intervaltree(rec, filt_func=lambda feat: feat.type in {'rRNA', 'tRNA'})
        for rec in dloader.load_genome(ref, path2.pubdata/'expression'/'DGY14PNAS'/'reference')
    }
    for ref in refs['reference_assembly']
}

In [4]:
workdir = path2.data/'expression'/'DGY14PNAS'

In [5]:
def calc_RNAseq_depth(tag):
    itvs = ignore_intervals[runs.set_index('tag').loc[tag, 'reference_assembly']]
    
    with pysam.AlignmentFile(workdir/'bam'/f'{tag}.bam', 'rb') as inbam:
        try:
            _, plus_tmp  = tempfile.mkstemp(suffix='.bam')
            _, minus_tmp = tempfile.mkstemp(suffix='.bam')

            with pysam.AlignmentFile(plus_tmp , 'wb', template=inbam) as plus_bam, \
                 pysam.AlignmentFile(minus_tmp, 'wb', template=inbam) as minus_bam :
                outbam = {True: minus_bam, False: plus_bam}
                for read in inbam:
                    # ignore unmapped or multi-mapped reads
                    if read.is_unmapped or read.has_tag('XS'): 
                        continue
                    # ignore reads that intersect with tRNA/rRNA on the same strand
                    rrn, rrs, rre = read.reference_name, read.reference_start, read.reference_end
                    if any((it.data.strand > 0) == read.is_reverse for it in itvs[rrn][rrs:rre]): 
                        continue

                    outbam[read.is_reverse].write(read)

            pos_idx = pd.MultiIndex.from_tuples([(ref, i) for ref, l in zip(inbam.references, inbam.lengths) for i in range(l)])

            with pysam.AlignmentFile(plus_tmp, 'rb') as plus_bam:
                depth_plus = pd.Series({
                    (pc.reference_name, pc.reference_pos): pc.nsegments
                    for pc in plus_bam.pileup()
                }, index=pos_idx, dtype=pd.Int64Dtype()).fillna(0)

            with pysam.AlignmentFile(minus_tmp, 'rb') as minus_bam:
                depth_minus = pd.Series({
                    (pc.reference_name, pc.reference_pos): pc.nsegments
                    for pc in minus_bam.pileup()
                }, index=pos_idx, dtype=pd.Int64Dtype()).fillna(0)
            
        except Exception as e:
            print(e)
        finally:
            os.remove(plus_tmp)
            os.remove(minus_tmp)
        
    
    depth_plus.to_pickle(workdir/'depth'/f'{tag}.plus.pkl.bz2')
    depth_minus.to_pickle(workdir/'depth'/f'{tag}.minus.pkl.bz2')

with Pool(40) as pool:
    for _ in tqdm(pool.imap_unordered(calc_RNAseq_depth, runs[runs['type']=='RNA']['tag']), total=3):
        pass

  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
def calc_Riboseq_density(tag):
    # Contrary to RNA-Seq, for Ribo-Seq reads, the sense strand of forward reads is + strand, while that of reverse reads is - strand.
    itvs = ignore_intervals[runs.set_index('tag').loc[tag, 'reference_assembly']]
    
    with pysam.AlignmentFile(workdir/'bam'/f'{tag}.bam', 'rb') as bamfile:
        depth_plus  = {ref: pd.Series(0, index=range(l)) for ref, l in zip(bamfile.references, bamfile.lengths)}
        depth_minus = {ref: pd.Series(0, index=range(l)) for ref, l in zip(bamfile.references, bamfile.lengths)}

        for read in bamfile:
            # ignore unmapped or multi-mapped reads
            if read.is_unmapped or read.has_tag('XS'): 
                continue
            # ignore reads that intersect with tRNA/rRNA on the same strand
            rrn, rrs, rre = read.reference_name, read.reference_start, read.reference_end
            if any((it.data.strand > 0) == read.is_reverse for it in itvs[rrn][rrs:rre]): 
                continue
                
            if read.is_reverse:
                depth_minus[rrn][rrs] += 1
            else:
                depth_plus[rrn][rre-1] += 1
        
        depth_plus  = pd.concat(depth_plus.values() , keys=depth_plus.keys() )
        depth_minus = pd.concat(depth_minus.values(), keys=depth_minus.keys())
        
        depth_plus.to_pickle(workdir/'depth'/f'{tag}.plus.pkl.bz2')
        depth_minus.to_pickle(workdir/'depth'/f'{tag}.minus.pkl.bz2')
    
    return depth_plus, depth_minus

with Pool(40) as pool:
    for _ in tqdm(pool.imap_unordered(calc_Riboseq_density, runs[runs['type']!='RNA']['tag']), total=4):
        pass

  0%|          | 0/4 [00:00<?, ?it/s]